In [1]:
# library

import os
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD


In [2]:
# feature select
# feature는 cell_type, sm_name, mean value, SMILES 이용, 
# 최대한 id_map에 맞게 최소한으로 설정할건데 일단은 de_train을 기준으로 해보자

de_train = pd.read_parquet("/home/aiuser/taeuk/open-problems-single-cell-perturbations/de_train.parquet")
id_map = pd.read_csv("/home/aiuser/taeuk/open-problems-single-cell-perturbations/id_map.csv")
submission = pd.read_csv("/home/aiuser/taeuk/open-problems-single-cell-perturbations/sample_submission.csv")

sm_name_id_map = sorted(id_map.sm_name.unique())
cell_type_id_map = sorted(id_map.cell_type.unique())

sm_name_de_train = sorted(de_train.sm_name.unique())
cell_type_de_train = sorted(de_train.cell_type.unique())

# cell type, compound dictionary 

cell_type_dict = {cell_type_de_train[i]:i for i in range(len(cell_type_de_train))}
sm_name_dict = {sm_name_de_train[i]:i for i in range(len(sm_name_de_train))}

# compound info by SMILES

smiles = list(de_train.SMILES.unique())
voc = []

r = re.compile(".")
for sm in smiles:
    voc += list(set(r.findall(sm)))
voc = list(set(voc))
voc.sort()
voc[:10]

df_smile = pd.DataFrame(np.zeros((len(smiles), len(voc))).astype(int))
df_smile.columns = voc
for i in range(df_smile.shape[0]):
    for ele in r.findall(smiles[i]):
        df_smile[ele][i] += 1
df_smile = pd.DataFrame(smiles).join(df_smile)
df_smile.columns = ["SMILES"] + list(df_smile.columns[1:])
sm_id = de_train.iloc[:, 1:4].drop("sm_lincs_id", axis=1).drop_duplicates()
df_smile = sm_id.merge(df_smile, how="left", on="SMILES")
df_smile.drop("SMILES", axis=1, inplace=True)

# build dataloader

class SCPset(Dataset):
    def __init__(self, dataset, df_smile, cell_type_dict, sm_name_dict):
        super(SCPset, self).__init__()
        if dataset is None:
            self.x = id_map.iloc[:, 1:]
            self.y = None
        else:
            self.x = dataset.iloc[:, :2]
            self.y = dataset.iloc[:, 5:]
        self.smile = df_smile
        self.cell_type_dict = cell_type_dict
        self.sm_name_dict = sm_name_dict
   
    def __getitem__(self, idx):
        cell, name = self.x.iloc[idx]
        x_cell = self.cell_type_dict[cell]
        x_comp = self.smile.loc[self.smile.sm_name==name, "#":].values[0]
        ele_idx = np.where(x_comp!=0)[0][1:]
        ele_val = x_comp[ele_idx]
        ele_idx = list(ele_idx) + [0]*(20-len(ele_idx))
        ele_val = list(ele_val) + [0]*(20-len(ele_val))
        x = [x_cell] + list(ele_idx) + list(ele_val)
        if self.y is None:
            return torch.tensor(x, dtype=torch.int64)
        else:
            y = self.y.iloc[idx, :]
            return torch.tensor(x, dtype=torch.int64),\
                    torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.x.shape[0]               

In [9]:
# Define model
'''
input : [cell, comp, smil]
x     : tensor size of (B, dim_embed)      
'''
        
class SCPmodel(nn.Module):
    def __init__(self, dim_embed, num_layers, num_adds, gene, device):
        super(SCPmodel, self).__init__()
        self.embed_cell = nn.Embedding(6, dim_embed//2, device=device)
        self.embed_comp = nn.Embedding(33, dim_embed//2, padding_idx=0, device=device)
        self.fc1 = nn.Linear(dim_embed, 256, device=device)
        self.fc2 = nn.Linear(256, 18211, device=device)
        self.relu = nn.ReLU()
        
            
    def forward(self, x):
        cell = self.embed_cell(x[:, 0])
        comp_idx = x[:, 1:21]
        comp_val = x[:, 21:]
        comp = self.embed_comp(comp_idx)
        comp = comp * comp_val.unsqueeze(2)
        comp = comp.mean(dim=1, keepdim=False)
        x = torch.cat([cell, comp], dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x

In [10]:
# Define util function

def MRRMSE(pred, y):
      pred = pred.detach().cpu().numpy()
      y = y.detach().cpu().numpy()
      return np.sqrt(np.square(y - pred).mean(axis=1)).mean()    

def mrrmse_loss(pred, y):
      return torch.sqrt(torch.square(pred - y).mean(dim=1)).mean()

def compose_loss(pred, y):
      return mrrmse_loss(pred, y) + F.smooth_l1_loss(pred, y)

def fix_random_seed(seed):
    #random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [11]:
# Define training function

def train_model(device, train_size, dim_model, num_layers, num_adds, genes, optimizer, criterion, learning_rate, num_epochs, batch_size, verbose):
      
      train_idx = np.random.choice(de_train.shape[0], int(de_train.shape[0]*train_size), replace=False)
      train_loader = DataLoader(SCPset(de_train.iloc[train_idx, :], df_smile, cell_type_dict, sm_name_dict),
                                batch_size=batch_size, shuffle=True)
      valid_loader = [1]
      if train_size < 1.:
            valid_idx = list(set(np.arange(de_train.shape[0])) - set(train_idx))
            valid_loader = DataLoader(SCPset(de_train.iloc[valid_idx, :], df_smile, cell_type_dict, sm_name_dict),
                                      batch_size=batch_size, shuffle=True)

      model = SCPmodel(dim_model, num_layers, num_adds, genes, device)
      
      if optimizer.lower()=="sgd":
            optimizer = SGD(list(model.parameters())+[model.embed_gene], lr=learning_rate, momentum=0.)
      elif optimizer.lower()=='adam':
            optimizer = Adam(model.parameters(), lr=learning_rate)
      else:
            print("Wrong optimizer!")
            return
      
      scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)
      
      if criterion.lower()=="l1":
            criterion = nn.SmoothL1Loss(0.01)
      elif criterion.lower()=="l2":
            criterion = nn.MSELoss()
      elif criterion.lower()=="MRRMSE":
            criterion = mrrmse_loss
      elif criterion.lower() == "compose":
            criterion = compose_loss
      else:
            print("Wrong criterion!")
            return
      
      best_mrrmse = 10
      for epoch in tqdm(range(1, num_epochs+1)):
            train_loss = 0.
            valid_loss = 0.
            train_mrrmse = 0.
            valid_mrrmse = 0.
            
            model.train()
            for x, y in train_loader:
                  x, y = x.to(device), y.to(device)
                  optimizer.zero_grad()
                  pred = model(x)
                  if F.mse_loss(pred[0,:], pred[1,:]).item() < 0.0001:
                        print("trivial solution !, mse of prediction is %.6f"%F.mse_loss(pred[0,:], pred[1,:]).item())
                  loss = criterion(pred, y)
                  #model.embed_gene.retain_grad()
                  loss.backward()
                  optimizer.step()
                  
                  train_loss += loss.item()
                  train_mrrmse += MRRMSE(pred, y)
            if train_size < 1.:
                  model.eval()
                  for x, y in valid_loader:
                        x, y = x.to(device), y.to(device)
                        with torch.no_grad():
                              pred = model(x)
                              loss = criterion(pred, y)
                        
                        valid_loss += loss.item()
                        valid_mrrmse += MRRMSE(pred, y)
            scheduler.step()      
            # print loss per epoch
            if verbose:
                  print("[Epoch : %2d] [Loss : %.4f / %.4f] [MRRMSE : %.3f / %.3f]"%(
                        epoch, train_loss/len(train_loader), valid_loss/len(valid_loader),
                        train_mrrmse/len(train_loader), valid_mrrmse/len(valid_loader)))
            if train_size < 1. and best_mrrmse > valid_mrrmse/len(valid_loader):
                  best_mrrmse = valid_mrrmse/len(valid_loader)
            
      return model, best_mrrmse

def infer_model(device, model):
      test_loader = DataLoader(SCPset(None, df_smile, cell_type_dict, sm_name_dict),
                               batch_size=255, shuffle=False)
      model.eval()
      for x in test_loader:
            x = x.to(device)
            with torch.no_grad():
                  pred = model(x)
      return pred

In [12]:
# training & prediction

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      
train_size = 1.
dim_model = 128
num_layers = 1
num_adds = 1 # 많이 더할수록(입력할수록) prediction이 유전자마다 비슷비슷해짐
learning_rate = 0.015
batch_size = 64
num_epochs = 100 # 10 으로 했을때 0.65

fix_random_seed(231111)
model, mrrmse = train_model(device, train_size, dim_model, num_layers, num_adds,
                            18211, "Adam", "L1", learning_rate, num_epochs, batch_size, True)
pred = infer_model(device, model)
data = pred.detach().cpu().numpy()
df = pd.DataFrame(data).reset_index()
df.columns = submission.columns
display(df.head())

gene_info = model.embed_gene.data.squeeze().cpu().numpy()
gene_info = pd.DataFrame(gene_info)
gene_info.index = submission.columns[1:]
cell_info = model.embed_cell.weight.detach().cpu().numpy()
cell_info = pd.DataFrame(cell_info)
cell_info.index = [k for k in cell_type_dict.keys()]
df_corr = cell_info.T.corr()
df_corr.columns = [k for k in cell_type_dict.keys()]
df_corr.index = [k for k in cell_type_dict.keys()]

display(gene_info)
display(cell_info)
display(df_corr)

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_1875206/2686368559.py:72: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  torch.tensor(y, dtype=torch.float32)


[Epoch :  1] [Loss : 0.5439 / 0.0000] [MRRMSE : 1.277 / 0.000]
[Epoch :  2] [Loss : 0.5232 / 0.0000] [MRRMSE : 1.243 / 0.000]
trivial solution !, mse of prediction is 0.000011
[Epoch :  3] [Loss : 0.5294 / 0.0000] [MRRMSE : 1.254 / 0.000]
[Epoch :  4] [Loss : 0.5323 / 0.0000] [MRRMSE : 1.252 / 0.000]
[Epoch :  5] [Loss : 0.5517 / 0.0000] [MRRMSE : 1.293 / 0.000]
[Epoch :  6] [Loss : 0.5188 / 0.0000] [MRRMSE : 1.239 / 0.000]
trivial solution !, mse of prediction is 0.000000
[Epoch :  7] [Loss : 0.5104 / 0.0000] [MRRMSE : 1.228 / 0.000]
[Epoch :  8] [Loss : 0.5014 / 0.0000] [MRRMSE : 1.213 / 0.000]
trivial solution !, mse of prediction is 0.000018
[Epoch :  9] [Loss : 0.5026 / 0.0000] [MRRMSE : 1.214 / 0.000]
[Epoch : 10] [Loss : 0.5179 / 0.0000] [MRRMSE : 1.230 / 0.000]
trivial solution !, mse of prediction is 0.000000
[Epoch : 11] [Loss : 0.5013 / 0.0000] [MRRMSE : 1.206 / 0.000]
[Epoch : 12] [Loss : 0.5003 / 0.0000] [MRRMSE : 1.205 / 0.000]
trivial solution !, mse of prediction is 0.0

TypeError: SCPset.__init__() missing 1 required positional argument: 'sm_name_dict'

In [ ]:
# save

title = f"scp_D{dim_model}_L{num_layers}_A{num_adds}_lr{learning_rate}_B{batch_size}_E{num_epochs}"
df.to_csv("/home/aiuser/taeuk/%s.csv"%title, header=True, index=False)
